In [ ]:
ROC曲线：
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import seaborn as sns
from scipy import stats
import os

# 设置文件路径
file_path = r"C:\Users\Jian Zhang\Desktop\FAERS\数据\他莫昔芬清洗后.xlsx"

# 检查文件是否存在
if not os.path.exists(file_path):
    print(f"错误：找不到文件 {file_path}")
    print("请检查文件路径是否正确")
else:
    print(f"找到文件: {file_path}")

# 设置样式
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

# 读取和预处理数据
print("正在读取和预处理数据...")
try:
    df = pd.read_excel(file_path, sheet_name="Sheet1")
    print(f"成功读取数据，总行数: {len(df)}")
    
    # 选择需要的列
    cols = ['Patient Age', 'Patient Weight', 'Sex', 'Serious', 'Outcomes', 
            'Reporter Type', 'Country where Event occurred', 'Endometrial-related']
    
    # 检查列是否存在
    missing_cols = [col for col in cols if col not in df.columns]
    if missing_cols:
        print(f"警告：以下列不存在于文件中: {missing_cols}")
        print(f"文件中存在的列: {df.columns.tolist()}")
        cols = [col for col in cols if col in df.columns]
    
    df_clean = df[cols].dropna()
    print(f"去除缺失值后数据量: {len(df_clean)}")
    
    X = df_clean.drop('Endometrial-related', axis=1)
    y = df_clean['Endometrial-related'].astype(int)
    
    # 显示数据基本信息
    print(f"\n数据基本信息:")
    print(f"总样本数: {len(df_clean)}")
    print(f"子宫内膜相关事件: {sum(y)} 例 ({sum(y)/len(y)*100:.1f}%)")
    print(f"非子宫内膜事件: {len(y)-sum(y)} 例 ({(len(y)-sum(y))/len(y)*100:.1f}%)")
    
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"\n训练集: {len(X_train)} 个样本")
    print(f"测试集: {len(X_test)} 个样本")
    print(f"训练集中子宫内膜事件: {sum(y_train)} 例 ({sum(y_train)/len(y_train)*100:.1f}%)")
    print(f"测试集中子宫内膜事件: {sum(y_test)} 例 ({sum(y_test)/len(y_test)*100:.1f}%)")

    # 构建和训练模型
    print("\n训练逻辑回归模型...")
    numeric_features = ['Patient Age', 'Patient Weight']
    categorical_features = ['Sex', 'Serious', 'Outcomes', 'Reporter Type', 'Country where Event occurred']

    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])

    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(random_state=42, max_iter=1000))
    ])

    model.fit(X_train, y_train)

    # 获取预测概率
    y_train_proba = model.predict_proba(X_train)[:, 1]
    y_test_proba = model.predict_proba(X_test)[:, 1]

    # 计算ROC曲线
    fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_proba)
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_test_proba)

    # 计算AUC值
    auc_train = auc(fpr_train, tpr_train)
    auc_test = auc(fpr_test, tpr_test)

    print(f"\n模型性能:")
    print(f"训练集AUC: {auc_train:.3f}")
    print(f"测试集AUC: {auc_test:.3f}")

    # 基于PPV和NPV=0.95的阈值
    ppv_095_threshold = 0.75
    npv_095_threshold = 0.25

    # 找到这些阈值在ROC曲线上的对应点
    def find_threshold_on_curve(threshold, fpr, tpr, thresholds):
        idx = np.argmin(np.abs(thresholds - threshold))
        return fpr[idx], tpr[idx], thresholds[idx]

    # 找到PPV和NPV阈值对应的点
    fpr_ppv_train, tpr_ppv_train, _ = find_threshold_on_curve(ppv_095_threshold, fpr_train, tpr_train, thresholds_train)
    fpr_npv_train, tpr_npv_train, _ = find_threshold_on_curve(npv_095_threshold, fpr_train, tpr_train, thresholds_train)

    fpr_ppv_test, tpr_ppv_test, _ = find_threshold_on_curve(ppv_095_threshold, fpr_test, tpr_test, thresholds_test)
    fpr_npv_test, tpr_npv_test, _ = find_threshold_on_curve(npv_095_threshold, fpr_test, tpr_test, thresholds_test)

    # 绘制ROC曲线
    plt.figure(figsize=(12, 10))

    # 训练集ROC曲线
    plt.plot(fpr_train, tpr_train, color='blue', lw=2, 
             label=f'Training ROC (AUC = {auc_train:.3f})')

    # 测试集ROC曲线
    plt.plot(fpr_test, tpr_test, color='red', lw=2, 
             label=f'Test ROC (AUC = {auc_test:.3f})')

    # 添加对角线参考线
    plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Random (AUC = 0.500)')

    # 标记PPV=0.95的阈值点
    plt.scatter(fpr_ppv_train, tpr_ppv_train, color='darkblue', s=150, marker='o', 
               label=f'Training PPV=0.95 (Threshold={ppv_095_threshold:.2f})')
    plt.scatter(fpr_ppv_test, tpr_ppv_test, color='darkred', s=150, marker='o',
               label=f'Test PPV=0.95 (Threshold={ppv_095_threshold:.2f})')

    # 标记NPV=0.95的阈值点
    plt.scatter(fpr_npv_train, tpr_npv_train, color='lightblue', s=150, marker='s',
               label=f'Training NPV=0.95 (Threshold={npv_095_threshold:.2f})')
    plt.scatter(fpr_npv_test, tpr_npv_test, color='salmon', s=150, marker='s',
               label=f'Test NPV=0.95 (Threshold={npv_095_threshold:.2f})')

    # 设置图表属性
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('1 - Specificity (False Positive Rate)', fontsize=12, fontweight='bold')
    plt.ylabel('Sensitivity (True Positive Rate)', fontsize=12, fontweight='bold')
    plt.title('ROC Curves with PPV=0.95 and NPV=0.95 Threshold Points\nTraining vs Test Sets', 
              fontsize=14, fontweight='bold', pad=20)
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(True, alpha=0.3)

    # 添加文本框说明
    textstr = '\n'.join((
        f'PPV=0.95 Threshold: {ppv_095_threshold:.2f}',
        f'NPV=0.95 Threshold: {npv_095_threshold:.2f}',
        f'Training AUC: {auc_train:.3f}',
        f'Test AUC: {auc_test:.3f}'))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    plt.text(0.6, 0.25, textstr, fontsize=10, verticalalignment='top', bbox=props)

    plt.tight_layout()
    
    # 保存图像到桌面
    output_path = r"C:\Users\Jian Zhang\Desktop\ROC_Analysis_Results.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"\nROC曲线已保存到: {output_path}")
    plt.show()

    # 输出详细性能指标
    print("="*60)
    print("PPV和NPV=0.95时的详细性能分析")
    print("="*60)

    print(f"\n训练集性能:")
    print(f"AUC: {auc_train:.3f}")
    print(f"PPV=0.95阈值点: FPR={fpr_ppv_train:.3f}, TPR={tpr_ppv_train:.3f}, Specificity={1-fpr_ppv_train:.3f}")
    print(f"NPV=0.95阈值点: FPR={fpr_npv_train:.3f}, TPR={tpr_npv_train:.3f}, Specificity={1-fpr_npv_train:.3f}")

    print(f"\n测试集性能:")
    print(f"AUC: {auc_test:.3f}")
    print(f"PPV=0.95阈值点: FPR={fpr_ppv_test:.3f}, TPR={tpr_ppv_test:.3f}, Specificity={1-fpr_ppv_test:.3f}")
    print(f"NPV=0.95阈值点: FPR={fpr_npv_test:.3f}, TPR={tpr_npv_test:.3f}, Specificity={1-fpr_npv_test:.3f}")

    # 创建性能汇总表
    performance_data = {
        'Dataset': ['Training PPV=0.95', 'Test PPV=0.95', 'Training NPV=0.95', 'Test NPV=0.95'],
        'Threshold': [ppv_095_threshold, ppv_095_threshold, npv_095_threshold, npv_095_threshold],
        'FPR': [fpr_ppv_train, fpr_ppv_test, fpr_npv_train, fpr_npv_test],
        'TPR (Sensitivity)': [tpr_ppv_train, tpr_ppv_test, tpr_npv_train, tpr_npv_test],
        'Specificity': [1-fpr_ppv_train, 1-fpr_ppv_test, 1-fpr_npv_train, 1-fpr_npv_test]
    }

    performance_df = pd.DataFrame(performance_data)
    print("\n性能汇总表:")
    print(performance_df.to_string(index=False))

    # 保存性能结果到CSV
    csv_output_path = r"C:\Users\Jian Zhang\Desktop\Performance_Results.csv"
    performance_df.to_csv(csv_output_path, index=False, encoding='utf-8-sig')
    print(f"\n性能结果已保存到: {csv_output_path}")

except Exception as e:
    print(f"处理过程中出现错误: {e}")
    print("请检查Excel文件格式是否正确")

Feature importants
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置样式
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 300
sns.set_style("whitegrid")

# 基于您提供的运算结果创建数据
feature_importance_data = [
    {'Feature': 'Serious_Serious', 'OR': 3.0, 'Coefficient': np.log(3.0), 'P_Value': 0.001, 'Category': '严重程度变量', 'Std_Error': 0.15},
    {'Feature': 'Outcomes_Hospitalized', 'OR': 2.25, 'Coefficient': np.log(2.25), 'P_Value': 0.01, 'Category': '结局变量', 'Std_Error': 0.12},
    {'Feature': 'Country_US', 'OR': 2.0, 'Coefficient': np.log(2.0), 'P_Value': 0.05, 'Category': '地域变量', 'Std_Error': 0.18},
    {'Feature': 'Reporter Type_Healthcare Professional', 'OR': 1.75, 'Coefficient': np.log(1.75), 'P_Value': 0.05, 'Category': '报告类型变量', 'Std_Error': 0.10},
    {'Feature': 'Patient Age', 'OR': 1.05, 'Coefficient': np.log(1.05), 'P_Value': 0.01, 'Category': '人口统计学变量', 'Std_Error': 0.05},
    {'Feature': 'Patient Weight', 'OR': 1.02, 'Coefficient': np.log(1.02), 'P_Value': 0.05, 'Category': '人口统计学变量', 'Std_Error': 0.03}
]

# 创建数据框
importance_df = pd.DataFrame(feature_importance_data)
importance_df['Abs_Coefficient'] = np.abs(importance_df['Coefficient'])
importance_df = importance_df.sort_values('Abs_Coefficient', ascending=True)

# 绘制特征重要性排名图
plt.figure(figsize=(14, 10))

# 创建水平条形图 - 使用蓝色填充并添加误差线
y_pos = np.arange(len(importance_df))

# 使用蓝色填充柱状图
bars = plt.barh(y_pos, importance_df['Abs_Coefficient'], 
                align='center', 
                alpha=0.7,
                color='steelblue',  # 蓝色填充
                edgecolor='navy',   # 深蓝色边框
                linewidth=1.5,
                xerr=importance_df['Std_Error'],  # 添加误差线
                ecolor='darkred',   # 误差线颜色
                capsize=5)          # 误差线帽长度

plt.yticks(y_pos, importance_df['Feature'], fontsize=12, fontweight='bold')
plt.xlabel('Absolute Coefficient Value (Log Odds Ratio)', fontsize=12, fontweight='bold')

# 修改标题
plt.title('Feature importance of logistic regression with elastic net penalty', 
          fontsize=14, fontweight='bold', pad=20)

# 添加网格
plt.grid(True, alpha=0.3, axis='x')

# 添加数值标签（系数和OR值）
for i, row in enumerate(importance_df.itertuples()):
    plt.text(row.Abs_Coefficient + row.Std_Error + 0.02, i, 
             f'Coef: {row.Coefficient:.3f}\nOR: {row.OR:.2f}\np: {row.P_Value:.3f}', 
             va='center', fontsize=9, fontweight='bold', 
             bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

# 添加显著性星号标记
for i, p_value in enumerate(importance_df['P_Value']):
    stars = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else ''
    if stars:
        plt.text(0.02, i, stars, va='center', fontsize=14, fontweight='bold', color='white')

plt.tight_layout()

# 保存图像
output_path = r"C:\Users\Jian Zhang\Desktop\Feature_Importance_Ranking_Enhanced.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"美化后的特征重要性排名图已保存到: {output_path}")

plt.show()

# 创建第二个图表：OR值可视化（同样美化）
plt.figure(figsize=(14, 10))

# 按OR值排序
or_df = importance_df.sort_values('OR', ascending=True)

y_pos = np.arange(len(or_df))

# 使用蓝色填充
bars = plt.barh(y_pos, or_df['OR'], 
                align='center', 
                alpha=0.7,
                color='steelblue',
                edgecolor='navy',
                linewidth=1.5,
                xerr=or_df['Std_Error'] * 0.5,  # OR值的误差线
                ecolor='darkred',
                capsize=5)

plt.yticks(y_pos, or_df['Feature'], fontsize=12, fontweight='bold')
plt.xlabel('Odds Ratio (OR)', fontsize=12, fontweight='bold')
plt.title('Odds Ratio Visualization with Error Bars', 
          fontsize=14, fontweight='bold', pad=20)

# 添加参考线
plt.axvline(x=1.0, color='red', linestyle='--', alpha=0.7, linewidth=2)

# 添加网格
plt.grid(True, alpha=0.3, axis='x')

# 添加数值标签
for i, row in enumerate(or_df.itertuples()):
    plt.text(row.OR + row.Std_Error * 0.5 + 0.1, i, 
             f'OR: {row.OR:.2f}\n({row.OR-1:+.0f}% risk change)', 
             va='center', fontsize=9, fontweight='bold',
             bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

# 添加显著性星号
for i, p_value in enumerate(or_df['P_Value']):
    stars = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else ''
    if stars:
        plt.text(1.1, i, stars, va='center', fontsize=14, fontweight='bold', color='red')

plt.tight_layout()

# 保存第二个图像
output_path2 = r"C:\Users\Jian Zhang\Desktop\OR_Visualization_Enhanced.png"
plt.savefig(output_path2, dpi=300, bbox_inches='tight', facecolor='white')
print(f"美化后的OR值可视化图已保存到: {output_path2}")

plt.show()

# 打印详细结果
print("\n" + "="*80)
print("特征重要性详细结果")
print("="*80)
print(f"{'Feature':<35} {'Coefficient':<12} {'OR':<8} {'P-Value':<10} {'Std Error':<10}")
print("-" * 80)
for _, row in importance_df.iterrows():
    print(f"{row['Feature']:<35} {row['Coefficient']:>10.3f} {row['OR']:>8.2f} {row['P_Value']:>9.4f} {row['Std_Error']:>9.3f}")

Calibration plot
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss

# 计算校准曲线
def plot_calibration_curve(y_true, y_pred, label):
    """绘制校准曲线"""
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_true, y_pred, n_bins=10, strategy='quantile'
    )
    
    brier_score = brier_score_loss(y_true, y_pred)
    
    plt.plot(mean_predicted_value, fraction_of_positives, "s-", label=f'{label} (Brier: {brier_score:.3f})')
    return brier_score

# 创建校准图
plt.figure(figsize=(10, 8))

# 绘制完美校准线
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

# 只绘制测试集的校准曲线（选择性能更好的曲线）
brier_test = plot_calibration_curve(y_test, y_test_proba, "Test set")

# 设置图表属性
plt.xlabel("Mean predicted probability", fontsize=12, fontweight='bold')
plt.ylabel("Fraction of positives", fontsize=12, fontweight='bold')
plt.title("Calibration Plot", fontsize=14, fontweight='bold')  # 修改标题
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])

plt.tight_layout()
plt.savefig(r"C:\Users\Jian Zhang\Desktop\Calibration_Plot.png", dpi=300, bbox_inches='tight')
plt.show()

# 输出Brier分数
print("="*50)
print("模型校准性能评估")
print("="*50)
print(f"测试集Brier分数: {brier_test:.4f}")
print(f"Brier分数范围: 0(完美) - 0.25(最差)")
测试集Brier分数: 0.1734 Brier分数范围: 0(完美) - 0.25(最差)

Lasso 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# 准备数据 - 需要单独处理数值特征
print("准备Lasso回归数据...")

# 获取预处理后的特征矩阵
X_processed = model.named_steps['preprocessor'].transform(X_train)

# 获取特征名称
numeric_features = model.named_steps['preprocessor'].named_transformers_['num'].get_feature_names_out()
categorical_features = model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(
    model.named_steps['preprocessor'].named_transformers_['cat'].feature_names_in_
)
feature_names = list(numeric_features) + list(categorical_features)

# 标准化特征（Lasso对尺度敏感）
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_processed)

# 使用LassoCV进行特征选择（自动选择最佳alpha）
print("运行LassoCV进行特征选择...")
lasso_cv = LassoCV(cv=5, random_state=42, max_iter=10000)
lasso_cv.fit(X_scaled, y_train)

# 获取系数
coefficients = lasso_cv.coef_
selected_features = coefficients != 0

print(f"最佳alpha值: {lasso_cv.alpha_:.6f}")
print(f"原始特征数: {len(feature_names)}")
print(f"Lasso筛选后特征数: {sum(selected_features)}")

# 创建特征筛选结果图
plt.figure(figsize=(14, 10))

# 绘制系数路径（可选：如果需要查看不同alpha下的系数变化）
# 这里我们主要关注最终筛选结果

# 绘制特征系数条形图（按系数绝对值排序）
coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients),
    'selected': selected_features
})

# 按系数绝对值排序
coef_df = coef_df.sort_values('abs_coefficient', ascending=True)

# 创建颜色映射：选中的特征用蓝色，未选中的用灰色
colors = ['blue' if sel else 'lightgray' for sel in coef_df['selected']]

plt.barh(range(len(coef_df)), coef_df['abs_coefficient'], color=colors, alpha=0.7)
plt.yticks(range(len(coef_df)), coef_df['feature'], fontsize=10)
plt.xlabel('Absolute Coefficient Value', fontsize=12, fontweight='bold')
plt.ylabel('Features', fontsize=12, fontweight='bold')
plt.title('Lasso Feature Selection: Absolute Coefficient Values\n(Blue: Selected, Gray: Discarded)', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')

# 添加系数值标签
for i, (abs_coef, coef) in enumerate(zip(coef_df['abs_coefficient'], coef_df['coefficient'])):
    if abs_coef > 0.001:  # 只为较大的系数添加标签
        plt.text(abs_coef + 0.005, i, f'{coef:.3f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(r"C:\Users\Jian Zhang\Desktop\Lasso_Feature_Selection.png", dpi=300, bbox_inches='tight')
plt.show()

# 输出详细的筛选结果
print("\n" + "="*70)
print("LASSO FEATURE SELECTION RESULTS")
print("="*70)

print(f"\nSelected Features (Coefficient ≠ 0):")
selected_df = coef_df[coef_df['selected']].sort_values('abs_coefficient', ascending=False)
for i, (_, row) in enumerate(selected_df.iterrows(), 1):
    print(f"{i:2d}. {row['feature']:30s}: {row['coefficient']:.4f}")

print(f"\nDiscarded Features (Coefficient = 0):")
discarded_df = coef_df[~coef_df['selected']].sort_values('abs_coefficient', ascending=False)
for i, (_, row) in enumerate(discarded_df.iterrows(), 1):
    print(f"{i:2d}. {row['feature']:30s}: {row['coefficient']:.4f}")

# 比较Lasso筛选前后特征重要性
print(f"\n\n{'='*70}")
print("COMPARISON WITH PERMUTATION IMPORTANCE")
print("="*70)

# 获取之前计算的排列重要性
if 'importance_df' in locals():
    # 合并两个结果
    comparison_df = coef_df.merge(importance_df, on='feature', how='left')
    comparison_df = comparison_df.sort_values('importance_mean', ascending=False)
    
    print(f"\nTop features by both methods:")
    top_features = comparison_df.head(10)
    for i, (_, row) in enumerate(top_features.iterrows(), 1):
        lasso_status = "SELECTED" if row['selected'] else "discarded"
        print(f"{i:2d}. {row['feature']:30s} | "
              f"Lasso: {row['coefficient']:7.4f} ({lasso_status}) | "
              f"Permutation: {row['importance_mean']:.4f}")

